In [ ]:
"""
10-meter aspen patch metrics
author: maxwell.cook@colorado.edu
"""



In [ ]:
fp = os.path.join(projdir,'Aim3/data/spatial/mod/future_fire_grid_trend.gpkg')
future_fire = gpd.read_file(fp)
future_fire.head()

In [ ]:
# check for duplicates, remove them
n = future_fire.duplicated(subset=['grid_id']).sum()
if n > 0:
    print(f"\nThere are [{n}] duplicate rows.\n")
else:
    print("\nNo duplicates at this stage.\n")

In [ ]:
# load the 10-m aspen map (classification)
aspen10_fp = os.path.join(projdir,'Aim1/data/spatial/mod/results/classification/s2aspen_distribution_10m_y2019_CookEtAl.tif')
aspen10 = rxr.open_rasterio(aspen10_fp, cache=False, chunks='auto', mask=True).squeeze()
print(f"\n{aspen10}\n")
print(aspen10.rio.crs)

In [ ]:
t0 = time.time()

# calculate zonal statistics
aspen10_grids = compute_band_stats(
    geoms=future_fire, 
    image_da=aspen10, 
    id_col='grid_id', 
    attr='aspen10',
    ztype='categorical'
)
# only keep the count of aspen pixels
aspen10_grids = aspen10_grids[aspen10_grids['aspen10'] == 1]
# check the results
print(aspen10_grids.head())

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.\n")
print("\n~~~~~~~~~~\n")

In [ ]:
aspen10_grids['pct_cover'].describe()

In [ ]:
# seperate aspen grids for patch analysis
aspen_grids = aspen10_grids[aspen10_grids['pct_cover'] > 1] # grids where aspen percent cover is greater than 1%
print(f"\n[{len(aspen10_grids)}/{len(future_fire)}] grids with any aspen cover.")
print(f"\n\t[{len(aspen_grids)}/{len(future_fire)}] grids with >1% aspen cover.\n")

In [ ]:
# merge back to the spatial data
ff_aspen_grids = future_fire[future_fire['grid_id'].isin(aspen_grids['grid_id'].unique())]
print(len(ff_aspen_grids))

In [ ]:
t0 = time.time()

# patch analysis - largest patch size, mean patch size, etc
import pylandstats as pls
from joblib import Parallel, delayed

# define metrics to calculate
cls_metrics = [
    'number_of_patches',  'patch_density', 'largest_patch_index'
]

def process_grid(grid):
    try:
        bounds = grid.geometry.bounds
        bbox = box(*bounds)

        # extract the raster for the gridcell:
        arr = aspen10.rio.clip([bbox])
        # generate the landscape
        ls = pls.Landscape(arr.values, res=(10,10))
        # calculate the patch metrics
        patches = ls.compute_class_metrics_df(metrics=cls_metrics)
        patches["grid_id"] = grid.grid_id
        del arr, ls
        
        return patches
        
    except Exception as e:
        print(f"Error processing grid {row.grid_id}: {e}")

metrics_ = [] # to store the patch metrics

# Parallel processing using joblib
num_cores = -1  # Use all available cores
N = len(ff_aspen_grids)
results = Parallel(n_jobs=num_cores)(
    delayed(process_grid)(grid) for _, grid in tqdm(ff_aspen_grids.iterrows(), total=N)
)
        
# merge the patch metrics
metrics_df = pd.concat(filter(None, results), ignore_index=True)
metrics_df.to_csv(os.path.join(projdir,"data/tabular/aspen_patch_metrics.csv"), index=False)

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.\n")
print("\n~~~~~~~~~~\n")